In [ ]:
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores.pinecone import Pinecone

from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import hub
from langchain.agents import Tool
from langchain.agents import AgentExecutor, create_json_chat_agent

from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AssemblyAIAudioTranscriptLoader
import assemblyai as aai
import pinecone
import os

In [ ]:
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENV'],
)

In [ ]:
aai_config = aai.TranscriptionConfig(
    speaker_labels=True, auto_chapters=True, entity_detection=True
)

In [ ]:
loader = AssemblyAIAudioTranscriptLoader(
    file_path="",
    config=aai_config,
    api_key=os.environ['ASSAMBLYAI_KEY'], )

In [ ]:
docs = loader.load()

In [ ]:
doc = docs[0]

metadata = doc.metadata
transcribed_text = 'Call Recording Id : %s' % (metadata['id']) + ' Call Recording Transcription : ' + doc.page_content
additional_context = ""

if metadata['chapters']:
    additional_context += "Call summary is detailed in the following section:"
    for chapter in metadata['chapters']:
        additional_context += "\nSummary: %s" % chapter['summary']
        additional_context += "\nHeadline: %s" % chapter['headline']
        additional_context += "\nGist: %s" % chapter['gist']
    additional_context += "\n\n"

if metadata['entities']:
    additional_context += "Entities involved in this call are detailed in the following section with the types:"
    for entity in metadata['entities']:
        additional_context += "\nType: %s" % entity['entity_type']
        additional_context += "\nName: %s" % entity['text']
    additional_context += "\n\n"


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)
docs_chunks = text_splitter.split_text(transcribed_text)

In [ ]:
index_name = metadata.get('id')
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPEN_AI_KEY'])
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, metric="cosine", dimension=1536)
docsearch = Pinecone.from_texts(docs_chunks, embeddings, index_name=index_name)

In [ ]:
llm = ChatOpenAI(
    temperature=0.9,
    openai_api_key=os.environ['OPEN_AI_KEY']
)

In [ ]:
retriever = docsearch.as_retriever(search_type='mmr', search_kwargs={
    'k': 2,
})

In [ ]:
PREFIX = """Answer the question using the given {tools}:"""
FORMAT_INSTRUCTIONS = """Follow this format:\n
Question: {question}\n
Action: your chosen action from [{tool_names}]\n
Action Input: your input for the action\n
Observation: the action's outcome\n"""
SUFFIX = """

Start!

Thought:{agent_scratchpad}

Answer:"" 
"""

TEMPLATE = PREFIX + FORMAT_INSTRUCTIONS + SUFFIX

prompt = PromptTemplate(
    input_variables=["question", "transcription"],
    template=TEMPLATE
)

In [ ]:
from langchain.chains import create_extraction_chain

# Schema
schema = {
    "properties": {
        "customer_name": {"type": "string"},
        "amount": {"type": "integer"},
        "agent_name": {"type": "string"},
        "partner_name": {"type": "string"},
    },
    "required": ["customer_name", "amount"],
}
chain = create_extraction_chain(schema, llm)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

combine_docs_chain = create_stuff_documents_chain(
    llm, prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
tools = [
    Tool(
        name="Summary",
        func=chain.run,
        description=(
            'use this tool to answer questions '
        )
    ),
    Tool(
        name='Data Retrival',
        func=retrieval_chain.invoke,
        description=(
            'use this tool to answer questions'
            'more information about the topic'
        )
    )
]

In [ ]:
from langchain.agents import create_react_agent

agent = create_react_agent(llm, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True,
    max_iterations=3,
    #memory = conversational_memory
)

In [ ]:
query = "What is the name of the customer?"
agent_executor.invoke(
    {"question": query, "transcription": transcribed_text, "additional_info": additional_context, 'context': CONTEXT})